In [16]:
#"Hayvan Tanıma Git Repostory Oluşturuldu"

In [1]:
import os
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, History
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.optimizers import SGD


os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"  # UYARILARI KAPATMA


/home/ufuk/anaconda3/envs/tensor/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ufuk/anaconda3/envs/tensor/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ufuk/anaconda3/envs/tensor/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ufuk/anaconda3/envs/tensor/lib/python3.7/site-

In [2]:
#print(os.listdir("./images/train"))
SIZE = 224
EPOCH = 50
BATCH = 32
train_directory = "images/train/"
valid_directory = "images/valid/"
test_directory = "images/test/"

data_aug = ImageDataGenerator(preprocessing_function=preprocess_input,
                              validation_split=0.3
                              )
test_aug = ImageDataGenerator(rotation_range=40,
                              width_shift_range=[0.2, 1.0],
                              height_shift_range=[0.2, 1.0],
                              shear_range=0.2,
                              rescale=1. / 255,
                              zoom_range=[0.2, 1.0],
                              horizontal_flip=True,
                              fill_mode="nearest",
                              vertical_flip=True,
                              brightness_range=[0.2, 1.0])
#Max batch size= available GPU memory bytes / 4 / (size of tensors + trainable parameters)
train_generator = data_aug.flow_from_directory(directory=train_directory,
                                               batch_size=BATCH,
                                               shuffle=True,
                                               class_mode="categorical",
                                               color_mode="rgb",
                                               target_size=(SIZE, SIZE),
                                               interpolation="nearest",
                                               subset="training"

                                               )
validation_generator = data_aug.flow_from_directory(directory=train_directory,
                                                    batch_size=BATCH,
                                                    shuffle=True,
                                                    class_mode="categorical",
                                                    color_mode="rgb",
                                                    target_size=(SIZE, SIZE),
                                                    interpolation="nearest",
                                                    subset="validation"
                                                    )
test_generator = data_aug.flow_from_directory(directory=test_directory,
                                              batch_size=1,
                                              shuffle=False,
                                              class_mode="categorical",
                                              target_size=(SIZE, SIZE)
                                              )




Found 4654 images belonging to 37 classes.
Found 1959 images belonging to 37 classes.
Found 777 images belonging to 37 classes.


In [3]:
print(train_generator.class_indices)

{'Abyssinian': 0, 'Bengal': 1, 'Birman': 2, 'Bombay': 3, 'British_Shorthair': 4, 'Egyptian_Mau': 5, 'Maine_Coon': 6, 'Persian': 7, 'Ragdoll': 8, 'Russian_Blue': 9, 'Siamese': 10, 'Sphynx': 11, 'american_bulldog': 12, 'american_pit_bull_terrier': 13, 'basset_hound': 14, 'beagle': 15, 'boxer': 16, 'chihuahua': 17, 'english_cocker_spaniel': 18, 'english_setter': 19, 'german_shorthaired': 20, 'great_pyrenees': 21, 'havanese': 22, 'japanese_chin': 23, 'keeshond': 24, 'leonberger': 25, 'miniature_pinscher': 26, 'newfoundland': 27, 'pomeranian': 28, 'pug': 29, 'saint_bernard': 30, 'samoyed': 31, 'scottish_terrier': 32, 'shiba_inu': 33, 'staffordshire_bull_terrier': 34, 'wheaten_terrier': 35, 'yorkshire_terrier': 36}


In [4]:
tf.keras.backend.clear_session()

In [5]:
InceptionV3_model = InceptionV3(input_shape=(SIZE, SIZE, 3), weights='imagenet', include_top=False)
for layer in InceptionV3_model.layers[:230]:
    layer.trainable = False
for layer in InceptionV3_model.layers[230:]:
    layer.trainable = True

Inception = tf.keras.Sequential([
    InceptionV3_model,
    Flatten(),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(37, activation='softmax'),

])
Inception.compile(optimizer=SGD(lr=0.0005, momentum=0.9), loss='categorical_crossentropy',
                  metrics=['acc']
                  )

inception_filepath = 'inceptionv3_' + '-saved-model-{epoch:02d}-loss-{loss:.2f}.hdf5'


for i, layer in enumerate(InceptionV3_model.layers):
    print(i, layer.name, layer.trainable)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
0 input_1 False
1 conv2d False
2 batch_normalization False
3 activation False
4 conv2d_1 False
5 batch_normalization_1 False
6 activation_1 False
7 conv2d_2 False
8 batch_normalization_2 False
9 activation_2 False
10 max_pooling2d False
11 conv2d_3 False
12 batch_normalization_3 False
13 activation_3 False
14 conv2d_4 False
15 batch_normalization_4 False
16 activation_4 False
17 max_pooling2d_1 False
18 conv2d_8 False
19 batch_normalization_8 False
20 activation_8 False
21 conv2d_6 False
22 conv2d_9 False
23 batch_normalization_6 False
24 batch_normalization_9 False
25 activation_6 False
26 activation_9 False
27 average_pooling2d False
28 conv2d_5 False
29 conv2d_7 False
30 conv2d_10 False
31 conv2d_11 False
32 batch_normalization_5 False
33 batch_normalization_7 False
34 batch_normalization_10 False
35 batch_normalization_11 False
36 activation_5 False
37 activation_7 

In [6]:


checkpoint = ModelCheckpoint(filepath=inception_filepath,
                             save_best_only=True,
                             monitor="val_acc",
                             mode="max",
                             verbose=1
                             )
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')

tensorboard = TensorBoard(log_dir="logs/{}".format(time()))
history_callback = History()

Step_size_train = train_generator.n // train_generator.batch_size
Step_size_valid = validation_generator.n // validation_generator.batch_size
Step_size_test = test_generator.n // test_generator.batch_size
print(Step_size_train, Step_size_valid)

inceptionv3_history = Inception.fit(train_generator, epochs=EPOCH,
                                    validation_data=validation_generator,
                                    callbacks=[history_callback,
                                               checkpoint,
                                               tensorboard, early],
                                    verbose=1)

145 61


In [ ]:
plt.plot(history_callback.history["acc"])
plt.plot(history_callback.history['val_acc'])
plt.legend(['accuracy', 'validation accuracy'])

In [ ]:
plt.plot(history_callback.history["loss"])
plt.plot(history_callback.history['val_loss'])
plt.legend(['accuracy', 'validation accuracy'])

In [ ]:
score, acc = Inception.evaluate_generator(generator=validation_generator,
                                                    steps=Step_size_valid)

test_generator.reset()

pred = Inception.predict_generator(generator=test_generator,
                                   steps=Step_size_test,
                                   verbose=1)
predicted_class_indces = np.argmax(pred, axis=1)

In [ ]:
labels = train_generator.class_indices
labels = dict((v, k) for k, v in labels.items())
predictions = [labels[k] for k in predicted_class_indces]

In [ ]:
filenames = test_generator.filenames
results = pd.DataFrame({"Filename": filenames,
                        "Predictions": predictions})
results.to_csv("results.csv", index=False)

In [ ]:
print("Test score ", score)
print("Test acc", acc)
results

In [7]:







#model = keras.models.load_model(filepath="inceptionv3_-saved-model-24-loss-0.01.hdf5")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [8]:
#model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 5, 5, 2048)        21802784  
_________________________________________________________________
flatten (Flatten)            (None, 51200)             0         
_________________________________________________________________
dense (Dense)                (None, 1024)              52429824  
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 37)                37925     
Total params: 74,270,533
Trainable params: 65,132,197
Non-trainable params: 9,138,336
_________________________________________________________________


In [10]:
#inceptionv3_keep= model.fit(train_generator, epochs=EPOCH, validation_data=validation_generator,   callbacks=[history_callback, checkpoint,tensorboard, early], verbose=1)

Epoch 1/50
145/146 [============================>.] - ETA: 0s - loss: 0.0108 - acc: 0.9976
Epoch 00001: val_acc improved from -inf to 0.86932, saving model to inceptionv3_-saved-model-01-loss-0.01.hdf5
146/146 [==============================] - 115s 791ms/step - loss: 0.0107 - acc: 0.9976 - val_loss: 0.5371 - val_acc: 0.8693
Epoch 2/50
145/146 [============================>.] - ETA: 0s - loss: 0.0108 - acc: 0.9976
Epoch 00002: val_acc improved from 0.86932 to 0.87187, saving model to inceptionv3_-saved-model-02-loss-0.01.hdf5
146/146 [==============================] - 95s 650ms/step - loss: 0.0108 - acc: 0.9976 - val_loss: 0.5352 - val_acc: 0.8719
Epoch 3/50
145/146 [============================>.] - ETA: 0s - loss: 0.0103 - acc: 0.9983
Epoch 00003: val_acc did not improve from 0.87187
146/146 [==============================] - 93s 636ms/step - loss: 0.0102 - acc: 0.9983 - val_loss: 0.5472 - val_acc: 0.8703
Epoch 4/50
145/146 [============================>.] - ETA: 0s - loss: 0.0127 - 

KeyboardInterrupt: 